In [5]:
import pandas as pd
import numpy as np
import zipfile
import os,re

from termcolor import colored

In [6]:
DICT_DIR = '../data/'
DICT_WIKI_PATH = 'dictionary_lowercase.zip'
DICT_WORD_PATH = 'wikipedia_en.zip'
DICT_MAIN_PATH = 'dict_main.zip'

In [7]:
dict_wiki = zipfile.ZipFile(DICT_DIR+DICT_WIKI_PATH)
dict_wiki = pd.read_csv(dict_wiki.open(re.sub('zip$','txt',DICT_WIKI_PATH)),header=None,names=['word'])
dict_wiki['word'] = dict_wiki['word'].apply(lambda x: str(x).upper())
dict_wiki = dict_wiki.drop_duplicates()

dict_word = zipfile.ZipFile(DICT_DIR+DICT_WORD_PATH)
dict_word = pd.read_csv(dict_word.open(re.sub('zip$','txt',DICT_WORD_PATH)),sep=' ',header=None,names=['word','len','instances','articles'])
dict_word['word'] = dict_word['word'].apply(lambda x: str(x).upper())
dict_word = dict_word.groupby(['word','len']).sum().sort_values(by='instances',ascending=False).reset_index()

In [8]:
dict_main = dict_word.merge(dict_wiki,how='inner',on='word',sort=False)

In [9]:
dict_main.to_csv(DICT_DIR+DICT_MAIN_PATH,compression=dict(method='zip',archive_name=re.sub('zip$','csv',DICT_MAIN_PATH)))

In [10]:
dict_main

,word,len,instances,articles
0,THE,3,32823802,4968822
1,OF,2,17445728,4279162
2,IN,2,15303927,4539682
3,AND,3,13537194,3886207
4,IS,2,8334346,4234968
...,...,...,...,...
54847,SCARIFY,7,2,2
54848,SCARILY,7,2,2
54849,SCANTINESS,10,2,2
54850,HENPECK,7,2,2


In [93]:
class Wordle:
    def __init__(self,data,first=True):
        if first is True:
            self.alldata = data
            self.breakword = 'IGIVEUP' #string to input if you can't get it and want to find the answer :)
            print('Welcome to \033[1mWORDLE!\n\033[0m')
        n = input('Please choose your desired word length : ')
        self.n = int(n)
        self.data = self.alldata[self.alldata['len']==self.n].reset_index(drop=True)
        self.word = self.data['word'][np.random.randint(len(self.data))]
        self.keys = {}

    # the n+1 keys save the state of each letter in your current guess, with the final one used to store the letter state in your guess history
    def set_keys(self,save_letters=False):
        for a in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
            if save_letters is False:
                self.keys[a] = list(np.zeros(self.n+1,dtype=int))
            else:
                self.keys[a] = list(np.zeros(self.n,dtype=int))+[self.keys[a][self.n]]

    def guess(self):
        t = 1
        # ansi escape codes for light grey, dark grey, orange and green
        colors = ['\033[38;5;254m','\033[38;5;239m','\033[38;5;214m','\033[38;5;36m']
        match = np.zeros(self.n,dtype=int)
        self.set_keys()
        while min(match) < 3:
            m = ''
            myguess = input(f'Guess {t} : ').upper()
            if myguess == self.breakword:
                print(f'The answer was {self.word} :)\n')
                match = [3 for i in match]
            if (len(myguess) != self.n) and (myguess != self.breakword):
                print(f'Please choose a {self.n}-letter word!')
                continue
            elif (re.match('^[A-Za-z]{'+str(self.n)+'}$',myguess) is None) and (myguess != self.breakword):
                print('Please use alphabetic characters only!')
                continue
            else:
                if myguess != self.breakword:
                    guessmap = []
                    for c in myguess:
                        guessmap.append(self.keys[c][0])
                        self.keys[c][0] += 1
                    self.set_keys(save_letters=True)
                    for i,c in enumerate(myguess):
                        if c == self.word[i]:
                            match[i] = self.keys[c][guessmap[i]-1] = 3
                        elif (c in self.word) and (c != self.word[i]):
                            match[i] = self.keys[c][guessmap[i]-1] = 2
                        else:
                            match[i] = self.keys[c][guessmap[i]-1] = 1
                        self.keys[c][self.n] = max(self.keys[c][self.n],match[i])
                        m += '\033[1m'+colors[match[i]]+c+' '+'\033[1m'
                    m += '\033[0m  :   '
                    for s in self.keys.keys():
                        m += '\033[1m'+colors[self.keys[s][self.n]]+' '+s+'\033[1m'
                    m += '\033[0m'
                    print(m)
                if min(match) == 3:
                    if myguess != self.breakword: print(f'You got it in {t} tries :)\n')
                    replay = input('Do you want to play again? y/n : ')
                    if replay.upper() == 'Y':
                        self.__init__(self.alldata,first=False)
                        self.guess()
                    else:
                        break
                else:
                    self.set_keys(save_letters=True)
                    t += 1

In [11]:
word = Wordle(dict_main)

In [13]:
word.guess()

In [95]:
if __name__ == '__main__':
    dict_main = pd.read_csv(zipfile.ZipFile(DICT_DIR+DICT_MAIN_PATH).open(re.sub('zip$','csv',DICT_MAIN_PATH)))
    word = Wordle(dict_main)
    word.guess()

In [9]:
import re
re.match('^[A-Za-z]{5}$','stars')

<re.Match object; span=(0, 5), match='stars'>

In [11]:
re.match('^[A-Za-z]{5}$','strars') is None

True